In [2]:
%matplotlib inline

import math
import sys
import os
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
import pandas as pd
import ibis
# from pyspark import SparkContext
# from pyspark.sql import SparkSession
# impala_client.close()
# hdfs_client.close()
ibis.options.sql.default_limit = None
impala_client = ibis.impala.connect(host='192.168.23.236', port=21050, user='app_bi')
hdfs_client = ibis.hdfs_connect(host='192.168.23.223', port=50070,user='app_bi').client

# spark = SparkSession.builder \
#    .master("yarn") \
#    .appName("spark_test") \
#       .config("spark.executor.instances", 20) \
#        .config("spark.executor.cores", 1) \
#        .config("spark.executor.memory", '4G') \
#        .config("spark.driver.memory", '4G') \
#    .enableHiveSupport() \
#    .getOrCreate()


In [4]:
_sql="""
        select
             id,
             name,
             parent_id,
             degree,
             root_id
        
        from odata.ods_tk_knowledge_hierarchy kh
        where
            grade_id='2'
            and new_subject_id='2'
    
""" 
df_kh = impala_client.sql(_sql).execute()


In [16]:
def search_kh(content, key):
    global g_kh_dict
    for node in g_kh_dict[key].itervalues():
        if node['name'] in content:
            yield node


def build_tree(records):
    # 知识点需要整理成树形结构

    kh_dict = {}
    tree = []
    kh_list = []
    for index,record in records.iterrows():
        record = record.to_dict()
        kh_dict[record['id']] = record
        # 根节点
        if record['root_id'] == '0' or record['id'] == record['parent_id'] or record['parent_id'] == '0':
            tree.append(record)
            continue
        kh_list.append(record)
    # kh_list = sorted(records, key=lambda x: x['degree'])
    # del records
    for record in kh_list:
        # record = kh_list.pop(0)
        # 没找到父节点，也作为根节点
        if record['parent_id'] not in kh_dict:
            tree.append(record)
            continue

        parent = kh_dict[record['parent_id']]
        parent.setdefault('childList', []).append(record)

    return tree

In [15]:
df_kh.iloc[0].to_dict()

{'degree': 4,
 'id': '0026cad84e364586bba7c692f1469a5c',
 'name': '\xe5\xa4\x9a\xe9\xa1\xb9\xe5\xbc\x8f\xe7\x9a\x84\xe5\x9f\xba\xe6\x9c\xac\xe6\xa6\x82\xe5\xbf\xb5',
 'parent_id': '6oo7ryi0e79zkqai08wbmeohxn6c03ac',
 'root_id': '6oo7ryi0e79zkqai08wbmeohxn6c03a7'}

In [17]:
trees=build_tree(df_kh)

In [37]:
def print_trees(trees):
    def print_tree(root):
        degree = root['degree']
        name = root['name']
        _id = root['id']
        line = [""] * 5
        line[0] = _id
        line[int(degree)] = name
        #line.append()
        yield line
        for node in root.get('childList', []):
            for line in print_tree(node):
                yield line

    for tree in trees:
        for line in print_tree(tree):
            yield line


data = {
    '0知识点ID': [],
    '1级': [],
    '2级': [],
    '3级': [],
    '4级': [],
    #'5级': [],
}
for line in print_trees(trees):
    data['0知识点ID'].append(line[0])
    data['1级'].append(line[1])
    data['2级'].append(line[2])
    data['3级'].append(line[3])
    data['4级'].append(line[4])
    #data['5级'].append(line[5])

import sys

reload(sys)
sys.setdefaultencoding('utf-8')
writer = pd.ExcelWriter("初中数学知识点.xlsx")
pd.DataFrame(data).to_excel(writer, encoding="UTF-8")
writer.save()


In [24]:
type(trees[0])

dict